In [ ]:
from qiskit.quantum_info import DensityMatrix, state_fidelity
import numpy as np


"""
def sqrt_pd_matrix(A : np.array) -> np.array:
   # Square root of positive definite matrix using eigendecomposition
    evals, evecs = np.linalg.eigh(A)
    #spectral theorem
    return sum( [ np.emath.sqrt(eigval) * np.outer(eigvec, np.conj(eigvec.T)) for eigval, eigvec in zip(evals, evecs) ] )
"""


def sqrt_pd_matrix(A : np.array) -> np.array:
    evals, evecs = np.linalg.eigh(A)
    return evecs @ np.diag(np.sqrt(np.maximum(evals, 1e-10))) @ evecs.T


def bures_distance(rho : np.array, sigma : np.array) -> float:
    # Compute fidelity between the two states

    rho = DensityMatrix(rho)
    sigma = DensityMatrix(sigma)

    F = state_fidelity(rho, sigma, validate=False)

    # Compute the Bures distance
    return np.arccos(np.sqrt(F))



def geodesic_density_matrix(rho : np.array, sigma :np.array, tau : float) -> np.array:

    #returns the geodesic between two states rho and sigma, given 
    #its density matrix np.array 


    theta_bures = bures_distance(rho, sigma) # theta term

    """
    if abs(theta_bures) < 1e-10:
        return rho if tau < 1e-10 else sigma if abs(tau - theta_bures) < 1e-10 else rho
    """


    sqrt_rho = sqrt_pd_matrix(rho) # √ρ 
    sqrt_sigma = sqrt_pd_matrix(sigma) # √σ
    
    # Calculate the amplitude term
    sqrt_sigma_rho = sqrt_sigma @ sqrt_rho # √σ√ρ
    abs_term = sqrt_pd_matrix(sqrt_sigma_rho.conj().T @ sqrt_sigma_rho) # |√σ√ρ| = √[( √σ√ρ )^† ( √σ√ρ ) ]

    # Calculate the inverse square root of rho
    inv_sqrt_rho = np.linalg.pinv(sqrt_rho, rcond=1e-10) # √ρ⁻¹
    
    # Calculate the cross-term (without the sin coefficients)
    cross_term = inv_sqrt_rho @ abs_term @ sqrt_rho # √ρ⁻¹ |√σ√ρ| √ρ
    

    # Now apply the formula directly
    sin2_theta = np.sin(theta_bures) ** 2 # sin²(θ)
    sin2_theta_minus_tau = np.sin(theta_bures - tau) ** 2 # sin²(θ - τ)
    sin2_tau = np.sin(tau) ** 2 # sin²(τ)
    sin_theta_minus_tau_sin_tau = np.sin(theta_bures - tau) * np.sin(tau) # sin(θ - τ) sin(τ)
    
    # First two terms
    result = (1 / sin2_theta) * (sin2_theta_minus_tau * rho + sin2_tau * sigma) # (sin²(θ - τ) ρ + sin²(τ) σ) / sin²(θ)
    
    # Third term + its Hermitian conjugate
    third_term = sin_theta_minus_tau_sin_tau * cross_term # sin(θ - τ) sin(τ) √ρ⁻¹ |√σ√ρ| √ρ
    result += (1 / sin2_theta) * (third_term + third_term.conj().T)
    
    # Ensure the result is a valid density matrix
    result = (result + result.conj().T) / 2  # Ensure perfect Hermiticity (only for numerical errors)
    return result

#lets try this:

# Pure states
ket_zero = np.array([1, 0, 0, 0])
ket_bell = np.array([1, 0, 0, 1]) / np.sqrt(2)

# Convert to density matrices
matrix_zerozero_state = DensityMatrix(ket_zero).data
matrix_bell_state = DensityMatrix(ket_bell).data

def parametrized_geodesic_zero_to_bell(tau):
    return geodesic_density_matrix(matrix_zerozero_state,
                                   matrix_bell_state,
                                     tau)



In [ ]:
import sympy as sp

# Define symbolic variable
t = sp.Symbol('t', real=True)

def symbolic_geodesic_density_matrix(rho, sigma, tau):
    """Computes the symbolic geodesic between rho and sigma."""
    n_decimals = 4

    theta_bures = sp.N( bures_distance(rho, sigma), n_decimals)  # Compute Bures distance symbolically
    
    sqrt_rho = sqrt_pd_matrix(rho)  # √ρ
    sqrt_sigma = sqrt_pd_matrix(sigma)  # √σ
    sqrt_sigma_rho = sqrt_sigma @ sqrt_rho  # √σ√ρ
    abs_term = sqrt_pd_matrix(sqrt_sigma_rho.conj().T @ sqrt_sigma_rho)  # |√σ√ρ|
    inv_sqrt_rho = np.linalg.pinv(sqrt_rho, rcond=1e-10)  # √ρ⁻¹
    
    # Compute the cross-term
    cross_term = inv_sqrt_rho @ abs_term @ sqrt_rho  # √ρ⁻¹ |√σ√ρ| √ρ
    
    # Symbolic expressions
    sin2_theta = sp.sin(theta_bures) ** 2
    sin2_theta_minus_tau = sp.sin(theta_bures - tau) ** 2
    sin2_tau = sp.sin(tau) ** 2
    sin_theta_minus_tau_sin_tau = sp.sin(theta_bures - tau) * sp.sin(tau)

    # Compute the matrix
    result = (1 / sin2_theta) * (sin2_theta_minus_tau * rho + sin2_tau * sigma)
    third_term = sin_theta_minus_tau_sin_tau * cross_term
    result += (1 / sin2_theta) * (third_term + third_term.conj().T)

    # Ensure Hermiticity
    result = (result + result.conj().T) / 2

    return sp.Matrix(result)  # Convert to SymPy matrix for symbolic manipulation

# Compute the symbolic geodesic matrix
symbolic_matrix = symbolic_geodesic_density_matrix(matrix_zerozero_state, matrix_bell_state, t)

# Convert to LaTeX
latex_representation = sp.latex(symbolic_matrix)
print(latex_representation)

In [ ]:
from IPython.display import display, Math
import sympy as sp

# Número de decimales a mostrar
n_decimals = 4

# Compute the symbolic geodesic matrix
symbolic_matrix = symbolic_geodesic_density_matrix(matrix_zerozero_state, matrix_bell_state, t)

# Truncate values
symbolic_matrix_truncated = symbolic_matrix.applyfunc(lambda x: sp.N(x, n_decimals))

# Display truncated symbolic matrix in Jupyter
display(symbolic_matrix_truncated)

# Display truncated LaTeX representation
display(Math(sp.latex(symbolic_matrix_truncated)))

In [ ]:
from qiskit.quantum_info import DensityMatrix, state_fidelity
import numpy as np
from scipy.linalg import sqrtm, pinvh

# Use float64 for maximum compatibility and precision
dtype = np.float64

def sqrt_pd_matrix(A: np.ndarray) -> np.ndarray:
    """
    Computes the square root of a positive semi-definite matrix using SciPy.
    """
    A = (A + A.conj().T) / 2  # Ensure Hermiticity
    return sqrtm(A).astype(dtype)

def inv_sqrt_matrix(A: np.ndarray) -> np.ndarray:
    """
    Computes the inverse square root of a positive semi-definite matrix using SciPy.
    """
    A = (A + A.conj().T) / 2  # Ensure Hermiticity
    sqrt_A = sqrtm(A).astype(dtype)

    # Perform the pseudoinverse
    return pinvh(sqrt_A).astype(dtype)


def bures_distance(rho: np.ndarray, sigma: np.ndarray) -> float:
    """
    Computes the Bures distance between two density matrices.
    """
    rho = DensityMatrix(rho)
    sigma = DensityMatrix(sigma)
    F = np.clip(state_fidelity(rho, sigma, validate=False), 0, 1)
    return np.arccos(np.sqrt(F))

def project_to_psd(A: np.ndarray) -> np.ndarray:
    """
    Projects a matrix to the nearest positive semi-definite matrix using eigen decomposition.
    """
    A = (A + A.conj().T) / 2
    evals, evecs = np.linalg.eigh(A)
    evals = np.clip(evals, 0, None)  # Set negative eigenvalues to zero
    return (evecs @ np.diag(evals) @ evecs.conj().T).astype(dtype)

def geodesic_density_matrix(rho: np.ndarray, sigma: np.ndarray, tau: float) -> np.ndarray:
    """
    Computes the geodesic density matrix using the Bures metric.
    """
    rho = rho.astype(dtype)
    sigma = sigma.astype(dtype)
    tau = np.float64(tau)
    
    theta_bures = bures_distance(rho, sigma)
    if np.abs(theta_bures) < 1e-10:
        return rho

    sqrt_rho = sqrt_pd_matrix(rho)
    sqrt_sigma = sqrt_pd_matrix(sigma)
    
    sqrt_sigma_rho = sqrt_sigma @ sqrt_rho
    abs_term = sqrt_pd_matrix(sqrt_sigma_rho.conj().T @ sqrt_sigma_rho)

    inv_sqrt_rho = inv_sqrt_matrix(rho)
    cross_term = inv_sqrt_rho @ abs_term @ sqrt_rho

    # Sine and cosine terms
    sin2_theta = np.sin(theta_bures) ** 2
    sin2_theta_minus_tau = np.sin(theta_bures - tau) ** 2
    sin2_tau = np.sin(tau) ** 2
    sin_theta_minus_tau_sin_tau = np.sin(theta_bures - tau) * np.sin(tau)
    
    # First two terms
    result = (1 / sin2_theta) * (sin2_theta_minus_tau * rho + sin2_tau * sigma)

    # Third term + its Hermitian conjugate
    third_term = sin_theta_minus_tau_sin_tau * cross_term
    result += (1 / sin2_theta) * (third_term + third_term.conj().T)

    # Ensure Hermiticity and positivity
    result = (result + result.conj().T) / 2
    result = project_to_psd(result)
    return result

# Pure states
ket_zero = np.array([1, 0, 0, 0])
ket_bell = np.array([1, 0, 0, 1]) / np.sqrt(2)

# Convert to density matrices
matrix_zerozero_state = DensityMatrix(ket_zero).data
matrix_bell_state = DensityMatrix(ket_bell).data

def parametrized_geodesic_zero_to_bell(tau):
    return geodesic_density_matrix(matrix_zerozero_state, matrix_bell_state, tau)


In [ ]:
from qiskit.visualization import plot_state_qsphere
from qiskit.quantum_info import DensityMatrix


theta = bures_distance(matrix_zerozero_state, matrix_bell_state)
tau_values = np.linspace(-10*theta, 10*theta, 2000) # a "continuos" of values of tau

geodesic_states = [parametrized_geodesic_zero_to_bell(tau) for tau in tau_values]

print(bures_distance(geodesic_states[0], geodesic_states[-1])) # should be 0, because they are the same state

In [ ]:

def is_pure_state(rho):
    # Check if it's a square matrix
    if rho.shape[0] != rho.shape[1]:
        return False
    
    # Check if trace equals 1 (within numerical precision)
    trace = np.trace(rho)
    if not np.isclose(trace, 1.0):
        return False
    
    # Check purity: Tr(ρ²) = 1 for pure states
    purity = np.trace(rho @ rho)
    
    print(purity)

    # Return True if purity is close to 1 (accounting for numerical error)
    return np.isclose(purity, 1.0)

# Here i check if the sates are puros
#spoiler si lo son 

for rho in geodesic_states:
   print(is_pure_state(rho))



print( 1/np.sin(theta) ** 2)